In [57]:
# coding: utf-8

# In[2]:


from __future__ import print_function
from functools import reduce
import re
import json

from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, LSTM, Dropout, Merge, Dense
from keras.models import Model, Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model 
from keras.models import model_from_json

# In[9]:
import os
import nltk
import string
from math import log
from nltk.corpus import wordnet as wn
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, RepeatVector
import gc


lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()


docs_corpus = []
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/documents.json",'r') as docs:
    docs_corpus = json.load(docs)

train_corpus = []
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/training.json",'r') as training:
    train_corpus = json.load(training)
print("len(train_corpus)",len(train_corpus))

test_for_train = train_corpus[:10000]

dev_corpus = []             
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/devel.json",'r') as devel:
    dev_corpus = json.load(devel)

test_for_del = dev_corpus[:200]

test_corpus = []             
with open("/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/testing.json",'r') as test:
    test_corpus = json.load(test)
    
test_for_predict = test_corpus[:10]

punc = string.punctuation
stopwordsAll = set(stopwords.words('english'))

def qestion_and_answer(question_id):
    question = train_corpus[question_id]['question']
    answer = train_corpus[question_id]['text']
    processed_question = word_tokenize(question)
    processed_answer = word_tokenize(answer)
    para_id = train_corpus[question_id]['answer_paragraph']
    docid = train_corpus[question_id]['docid']
    # print("processed_question",processed_question)
    # print("answer",answer)
    # print("docid",docid)
    return processed_question, processed_answer, para_id,docid


# qestion_and_answer(0)

def doc_to_story(para_id,docid):
    story = []

    doc = docs_corpus[docid]
    para = ""
    for index, para_data in enumerate(doc['text']):
        if index == para_id:
            para = para_data
        sents = sent_tokenize(para)
        # print("sents",sents)
        for sent in sents:
            tokens = word_tokenize(sent)
            # print("tokens",tokens)
            story.append(tokens)
    # print("story",story)
    return story


# doc_to_story(0)

# save [(story, question, answer)]
def prepare_data(train_corpus):
    final_data = []
    print(len(train_corpus))
    for i in range(len(train_corpus)):
        processed_question, answer, para_id,docid = qestion_and_answer(i)
        story = doc_to_story(para_id,docid)
        final_data.append((story, processed_question, answer))
        # print("final_data",final_data)
    return final_data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        story_list = []
        for sent in story:
            for w in sent:
                story_list.append(w)
        x = [word_idx[w] for w in story_list]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        for token in answer:
            y[word_idx[token]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), np.array(ys)


def get_vocab(data,del_data,test_data):
    vocab = set()
    for story, q, answer in data:
        list_words = []
        for sent in story:
            list_words = list_words + sent
        vocab_list = list_words + q + answer
        # print("vocab_list",vocab_list)
        vocab |= set(vocab_list)
    for story, q, answer in del_data:
        list_words = []
        for sent in story:
            list_words = list_words + sent
        vocab_list = list_words + q + answer
        # print("vocab_list",vocab_list)
        vocab |= set(vocab_list)
    for story, q in test_data:
        list_words = []
        for sent in story:
            list_words = list_words + sent
        vocab_list = list_words + q
        # print("vocab_list",vocab_list)
        vocab |= set(vocab_list)
    vocab = sorted(vocab)
    return vocab

data = prepare_data(test_for_train)
print("len(data)", len(data))


def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    ys = []
    for story, query, answer in data:
        story_list = []
        for sent in story:
            for w in sent:
                story_list.append(w)
        x = [word_idx[w] for w in story_list]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        for w in answer:
            y[word_idx[w]] = 1
        xs.append(x)
        xqs.append(xq)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), np.array(ys)


def get_paragraph(docid, document_data):
    # get the paragraph that contains the answer
    for item in document_data:
        if item['docid'] == docid:
            document = item['text']
            break
    return document


def term_freqs(document):
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for sentence in document:
        for token in word_tokenize(sentence):
            if token not in stopwordsAll and token not in punc:
                term = lemmatize(token.lower())
                tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1
                tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1
        doc_id += 1
    return tfs, doc_id + 1, tfs_forward


def get_okapibm25(tf, total_docment, documents):
    '''Calculate and return term weights based on okapibm25'''
    k1, b, k3 = 1.5, 0.5, 0
    okapibm25 = defaultdict(dict)

    # calculate average doc length
    total = 0
    for d in documents:
        total += len(d)
    avg_doc_length = total / len(documents) * 1.0

    for term, doc_list in tf.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items():
            # term occurences in query
            # qtf = question.count(term) # SEPCIAL
            qtf = 1.2
            idf = log((total_docment - df + 0.5) / df + 0.5)
            tf_Dt = ((k1 + 1) * tf[term][doc_id]) / (
            k1 * ((1 - b) + b * (len(documents[doc_id]) / avg_doc_length) + tf[term][doc_id]))
            if qtf == 0:
                third = 0
            else:
                third = ((k3 + 1) * qtf) / (k3 + qtf)
                okapibm25[term][doc_id] = idf * tf_Dt * third

    return okapibm25


# find top_k paragraph that may contain the answer
def get_top_k_document(tfidf, query, k, document):
    top_document_id = Counter()
    for token in word_tokenize(query):
        if token not in stopwordsAll:
            term = lemmatizer.lemmatize(token.lower())
            term_tfidf = tfidf[term]
            for docid, weight in term_tfidf.items():
                top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(k)
    top_document = []
    for document_id, weight in top_document_id:
        top_document.append(document_id)
    return top_document

def prepare_test_del(train_corpus):
    final_data = []
    print(len(train_corpus))
    for i in range(len(train_corpus)):
        story = []
        question = train_corpus[i]['question']
        processed_question, answer, para_id, docid = qestion_and_answer(i)
        document = get_paragraph(docid, docs_corpus)
        tfs, total_docment, tfs_forward = term_freqs(document)
        tfidf = get_okapibm25(tfs, total_docment, document)
        top_1 = get_top_k_document(tfidf, question, 1, document)
        for item in top_1:
            # print("item", item)
            story = doc_to_story(item, docid)
            # print("story",story)
        final_data.append((story, processed_question, answer))
        # print("final_data",final_data)
    return final_data

def prepare_test_data(test_corpus):
    final_data = []
    for item in test_corpus:
        question = item['question']
        docid = item['docid']
        processed_question = word_tokenize(question)
        document = get_paragraph(docid,docs_corpus)
        tfs,total_docment,tfs_forward = term_freqs(document)
        tfidf = get_okapibm25(tfs, total_docment,document)
        top_1 = get_top_k_document(tfidf,question,1,document)
        for item in top_1:
            #print("top_1", top_1)
            story = doc_to_story(item, docid)
            #print("story",story)
        final_data.append((story, processed_question))
        #print("final_data",final_data)
    return final_data

del_data = prepare_test_del(test_for_del)
print("len(del_data)", len(del_data))

def vectorize_stories_test(data, word_idx, story_maxlen, query_maxlen):
    xs = []
    xqs = []
    for story, query in data:
        story_list = []
        for sent in story:
            for w in sent:
                story_list.append(w)
        x = [word_idx[w] for w in story_list]
        xq = [word_idx[w] for w in query]
        xs.append(x)
        xqs.append(xq)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen)

print("len(test_for_predict)",len(test_for_predict))
test_data = prepare_test_data(test_for_predict)
print("len(test_data)",len(test_data))

test_for_pre = dev_corpus[:200]
prediction_data = prepare_test_data(test_for_pre)

vocab = get_vocab(data,del_data,prediction_data)
# print("vocab",vocab)
print("len(vocab)", len(vocab))



RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
WORD2VEC_EMBED_SIZE = 100
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 40

WORD2VEC_EMBED_SIZE = 100
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
word2idx = dict((c, i + 1) for i, c in enumerate(vocab))
idx2word = dict((i + 1, c) for i, c in enumerate(vocab))
print("len(word2idx)",len(word2idx))
story_maxlen = max(map(len, (x for x, _, _ in data)))
query_maxlen = max(map(len, (x for _, x, _ in data)))

x, xq, y = vectorize_stories(data, word2idx, story_maxlen, query_maxlen)
tx, txq, ty = vectorize_stories(del_data, word2idx, story_maxlen, query_maxlen)

print("Prepare test data...")

#test_story, test_question = vectorize_stories_test(test_data, word2idx, story_maxlen, query_maxlen)
#test_question_data = [test_story, test_question]

test_story_pre, test_question_pre = vectorize_stories_test(prediction_data, word2idx, story_maxlen, query_maxlen)
test_question_pre = [test_story_pre, test_story_pre]

del data
del del_data
gc.collect()

#word2vec
modelname = "/Users/shimei/Documents/2018/Web_Search/homework/Final_Assignment/project_files/mywvmodel.model"
model = Word2Vec.load(modelname)  # 3个文件放在一起：Word60.model   Word60.model.syn0.npy   Word60.model.syn1neg.npy
print("read model successful")
embedding_weights = np.zeros((vocab_size, WORD2VEC_EMBED_SIZE))
for word, index in word2idx.items():
    try:
        embedding_weights[index, :] = model[word.lower()]
    except KeyError:
        pass  # keep as zero (not ideal, but what else can we do?)

del model
del word2idx
gc.collect()

#former

# print('vocab = {}'.format(vocab))
print('x.shape = {}'.format(x.shape))
print('xq.shape = {}'.format(xq.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))




len(train_corpus) 43379
10000
len(data) 10000
200
len(del_data) 200
len(test_for_predict) 10
len(test_data) 10
len(vocab) 42516
RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100
len(word2idx) 42516
Prepare test data...
read model successful


/Users/shimei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:372: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


x.shape = (10000, 1305)
xq.shape = (10000, 60)
y.shape = (10000, 42517)
story_maxlen, query_maxlen = 1305, 60


In [54]:
'''
print('Build model...')

qenc = Sequential()
qenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_weights], mask_zero=True))
qenc.add(LSTM(EMBED_HIDDEN_SIZE, input_shape=(None,story_maxlen), return_sequences=False))
qenc.add(Dropout(0.3))

aenc = Sequential()
aenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_weights], mask_zero=True))
aenc.add(LSTM(EMBED_HIDDEN_SIZE, input_shape=(None,query_maxlen), return_sequences=False))
#aenc.add(RepeatVector(story_maxlen))
aenc.add(Dropout(0.3))

model = Sequential()
model.add(Merge([qenc, aenc], mode="sum"))
model.add(Dense(vocab_size, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])



print('Training')

model.fit([x, xq], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txq], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

print("Saving model...")
# serialize model to JSON
model_json = model.to_json()
with open("predictor_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("predictor_model.h5")
print("Saved model to disk")
 
# later...
'''

'\nprint(\'Build model...\')\n\nqenc = Sequential()\nqenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,\n                   weights=[embedding_weights], mask_zero=True))\nqenc.add(LSTM(EMBED_HIDDEN_SIZE, input_shape=(None,story_maxlen), return_sequences=False))\nqenc.add(Dropout(0.3))\n\naenc = Sequential()\naenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,\n                   weights=[embedding_weights], mask_zero=True))\naenc.add(LSTM(EMBED_HIDDEN_SIZE, input_shape=(None,query_maxlen), return_sequences=False))\n#aenc.add(RepeatVector(story_maxlen))\naenc.add(Dropout(0.3))\n\nmodel = Sequential()\nmodel.add(Merge([qenc, aenc], mode="sum"))\nmodel.add(Dense(vocab_size, activation="softmax"))\n\nmodel.compile(optimizer="adam", loss="categorical_crossentropy",\n              metrics=["accuracy"])\n\n\n\nprint(\'Training\')\n\nmodel.fit([x, xq], y,\n          batch_size=BATCH_SIZE,\n          epochs=EPOCHS,\n          validation_split=0.05)\nlo

In [58]:
# load json and create model
json_file = open('predictor_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("predictor_model.h5")
print("Loaded model from disk")


outputs = loaded_model.predict(test_question_pre, batch_size = 1)

'''
print('Testing...')
outputs = loaded_model.predict(test_question_data, batch_size = 1)
print("type(outputs)",type(outputs))
'''



/Users/shimei/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1271: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Loaded model from disk


'\nprint(\'Testing...\')\noutputs = loaded_model.predict(test_question_data, batch_size = 1)\nprint("type(outputs)",type(outputs))\n'

In [65]:
import spacy
from __future__ import unicode_literals
from bs4 import BeautifulSoup
nlp = spacy.load('en_core_web_sm')

print("outputs",outputs)

def get_top_k_document_text(tfidf, query, k, document):
    top_document_id = Counter()
    for token in word_tokenize(query):
        #print("token",token)
        if token not in stopwordsAll:
            term = lemmatizer.lemmatize(token.lower())
            #print("term",term)
            term_tfidf = tfidf[term]
            #print("term_tfidf",term_tfidf)
            for docid, weight in term_tfidf.items():
                top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(k)
    #print("top_document_id",top_document_id)
    top_document = []
    for document_id, weight in top_document_id:
        top_document.append(document[document_id])
        #print("top_document",top_document)
    if top_document == []:
        print("top_document is null.")
        top_document.append(document[2])
    #print("top_document",top_document)
    return top_document

def prepare_process_data(test_corpus, index):
    final_data = []
    question = test_corpus[index]['question']
    docid = test_corpus[index]['docid']
    processed_question = word_tokenize(question)
    document = get_paragraph(docid,docs_corpus)
    tfs,total_docment,tfs_forward = term_freqs(document)
    tfidf = get_okapibm25(tfs, total_docment,document)
    top_1 = get_top_k_document_text(tfidf,question,1,document)
    #print("top_1",top_1)
    final_data.append(top_1)
    #print("final_data",final_data)
    return final_data

count = 0
predictions = []
for index,answer_id in enumerate(outputs):
    #print(index, answer_id)
    try:
        story = prepare_process_data(test_for_del,index)
    except:
        continue
    print("story",len(story[0]))
    doc = ""
    for sents in story:
        #print("sents",sents)
        for sent in sents:
            #print("sent",sent)
            doc = nlp(sent) 
    id_list = []
    for item in answer_id:
        id_list.append(item)
    index1 = np.argmax(id_list)
        #print("type(number)",type(number))
        #print("number",number)
    try:
        text1 = idx2word[index1]
    except:
        continue
    print("index",index,"text1",text1)
    answer_str = (index,text1)
    predictions.append(answer_str)
    count += 1
print("count",count)




outputs [[9.4265751e-09 7.6395110e-05 9.4028241e-05 ... 9.4338599e-09
  9.1478665e-09 9.8371205e-09]
 [3.1469949e-07 1.0241414e-05 2.2445070e-04 ... 3.1329145e-07
  3.0461254e-07 3.2629580e-07]
 [7.4053497e-07 1.0680603e-03 4.9329636e-04 ... 7.3408154e-07
  7.2627756e-07 7.5305229e-07]
 ...
 [1.1436958e-06 9.4233743e-05 1.2840941e-05 ... 1.1284994e-06
  1.1328924e-06 1.1675074e-06]
 [2.1749463e-07 1.6511841e-04 3.1975785e-04 ... 2.1468661e-07
  2.1066796e-07 2.2280359e-07]
 [3.2122493e-08 2.7648508e-05 2.4418243e-05 ... 3.2006021e-08
  3.1178910e-08 3.3256946e-08]]
story 1
index 0 text1 salon
story 1
index 1 text1 purpose
story 1
index 2 text1 holocaust
story 1
story 1
index 4 text1 23.2
story 1
index 5 text1 AOL
story 1
index 6 text1 morale
story 1
index 7 text1 salon
story 1
story 1
index 9 text1 purpose
story 1
index 10 text1 salon
story 1
story 1
index 12 text1 salon
story 1
index 13 text1 23.2
story 1
index 14 text1 morale
story 1
index 15 text1 salon
story 1
index 16 text1 holoca

KeyboardInterrupt: 

In [64]:
import csv
headers = ['id', 'answer']
print("predictions[0]",predictions[0])
print("len(predictions)",len(predictions))
rows = predictions
with open('output_zz.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)
    
print("Finishing writing into output.csv!")



predictions[0] (0, 'salon')
len(predictions) 187
Finishing writing into output.csv!
